In [1]:
from emutils.imports import *
from emutils.utils import load_pickle

from utils import *

Python  3.6.13
Python Executable: /home/ubuntu/anaconda3/envs/cfshap22/bin/python
CWD =  /home/ubuntu/air/xai/cfshap-release/cf-shap-facct22
NumPy 1.19.2 | Pandas 1.1.5 | SciPy 1.5.2 | NetworkX 2.5.1 | StatsModels 0.12.2
scikit-learn 0.24.2 | xgboost 1.3.3 
MatPlotLib 3.3.4 | Seaborn 0.11.1 | 

In [2]:
from constants import DATA_DIR, MODEL_DIR, RESULTS_DIR

parser = ArgumentParser(sys.argv)

# General
parser.add_argument('--data_path', type=str, default=DATA_DIR, required=False)
parser.add_argument('--results_path', type=str, default=RESULTS_DIR, required=False)
parser.add_argument('--model_path', type=str, default=MODEL_DIR)

args, unknown = parser.parse_known_args()
args = attrdict(vars(args))

# Data

In [3]:
results = []
for dataset, data_version, model_version in tqdm([
        ('heloc', 'v2', 'v5'), 
        ('lendingclub', 'v2', 'v5'), 
        ('wines', 'v2', 'v5'), 
    ]):
    results.extend(load_pickle(profiling_filename(args, dataset, data_version, model_version, model_type = 'xgb')))

  0%|          | 0/3 [00:00<?, ?it/s]

[2022-09-11 23:14:34] INFO:	 Loading PICKLE from ./results/heloc_Dv2Mv5_xgb_profiling.pkl...
[2022-09-11 23:14:34] INFO:	 Load done in 0.0005 seconds.
[2022-09-11 23:14:34] INFO:	 Loading PICKLE from ./results/lendingclub_Dv2Mv5_xgb_profiling.pkl...
[2022-09-11 23:14:34] INFO:	 Load done in 0.0003 seconds.
[2022-09-11 23:14:34] INFO:	 Loading PICKLE from ./results/wines_Dv2Mv5_xgb_profiling.pkl...
[2022-09-11 23:14:34] INFO:	 Load done in 0.0004 seconds.


# Results

Let's check the results

In [4]:
from emutils.utils import pandas_max
pandas_max(200, 200)

In [5]:
def condition(m):
    if m in ['training_100', 'diff_pred_100', 'diff_label_100', 'knn100_qL1', 'knn10_qL1']:
        return True
    return False

results_ = [r for r in results if condition(r['Method'])]

df = pd.DataFrame(results_)
del df['Runtimes'], df['Model']
for col in ['Mean','Minimum', 'Maximum','Variance']:
    df[col] = df[col] .apply(lambda x : f"${x * 1e6:,.0f} \mu s$")

df = df[['Method', 'Dataset', 'Minimum']]
df = df.sort_values(['Dataset'])
df['Method'] = df['Method'].apply(method_to_name)
df = df[df['Dataset'] != 'Toy Moons']


df = df.pivot(index = 'Method', columns = 'Dataset')
df.columns = df.columns.droplevel()

map_cols = {
    'GMSC': 'GMSC',
    'HELOC': 'HELOC',
    'Lending': 'LC',
    'Wine': 'WQ',
}

def map_colsf(c):
    for a, b in map_cols.items():
        if a in c:
            return b

df = df.rename(columns = {col: map_colsf(col) for col in df})

df.T

Method,CF-SHAP $10$-NNQ+L1,CF-SHAP $100$-NNQ+L1,SHAP D-LAB (n = 100),SHAP D-PRED (n = 100),SHAP TRAIN (n = 100)
Dataset,,,,,
HELOC,$197 \mu s$,$730 \mu s$,$667 \mu s$,$642 \mu s$,$417 \mu s$
LC,$424 \mu s$,"$2,738 \mu s$","$2,977 \mu s$","$2,971 \mu s$","$2,220 \mu s$"
WQ,$168 \mu s$,$632 \mu s$,$633 \mu s$,$625 \mu s$,$391 \mu s$


In [6]:
print(df.to_latex(bold_rows=False, escape = False, column_format='rllll')
      .replace('toprule', 'hline')
      .replace('midrule', 'hline')
      .replace('\\bottomrule\n', '')
      .replace('Dataset', '')
      .replace('Method', '')
)

\begin{tabular}{rllll}
\hline
 &        HELOC &             LC &           WQ \\
                &              &                &              \\
\hline
CF-SHAP $10$-NNQ+L1   &  $197 \mu s$ &    $424 \mu s$ &  $168 \mu s$ \\
CF-SHAP $100$-NNQ+L1  &  $730 \mu s$ &  $2,738 \mu s$ &  $632 \mu s$ \\
SHAP D-LAB (n = 100)  &  $667 \mu s$ &  $2,977 \mu s$ &  $633 \mu s$ \\
SHAP D-PRED (n = 100) &  $642 \mu s$ &  $2,971 \mu s$ &  $625 \mu s$ \\
SHAP TRAIN (n = 100)  &  $417 \mu s$ &  $2,220 \mu s$ &  $391 \mu s$ \\
\end{tabular}

